In [1]:
import json
import numpy as np
import pandas as pd
import keras
import os
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import text_to_word_sequence
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Activation, Dropout, Input, Flatten, Embedding, LSTM
from keras.utils import np_utils
from keras.utils import to_categorical
from sklearn import preprocessing
import sklearn
import time
from random import shuffle
import pickle

c:\users\kevin\anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
os.getcwd()

'E:\\GT\\JrDesign\\demo\\grocery_e-receipt\\approaches\\LSTM'

In [3]:
with open("../../synthetic_data/data/final_train_labels.json") as f:
    data = json.load(f)

In [4]:
x = [ea[1] for ea in data]
y = [ea[0] for ea in data]
df = pd.DataFrame(data={"X": x, "Y": y})
print (len(df))
df.head()

528610


,X,Y
0,MXCN SHRDDD CHS BLND,MEXICAN SHREDDED CHEESE BLEND
1,VGTBL SHRTNNG,VEGETABLE SHORTENING
2,SQD,SQUID
3,SLCD MNZNLL LVS SLD,SLICED MANZANILLA OLIVES SALAD
4,PNPPL CHNKS TS WN JC,PINEAPPLE CHUNKS ITS OWN JUICE


In [5]:
# Y Vocab
labels = y
le = preprocessing.LabelEncoder()
df['encoded_y'] = le.fit_transform(y)
print ("# categories = ", max(df.encoded_y.tolist())+1)
df.head(5)

# categories =  105722


,X,Y,encoded_y
0,MXCN SHRDDD CHS BLND,MEXICAN SHREDDED CHEESE BLEND,58983
1,VGTBL SHRTNNG,VEGETABLE SHORTENING,100452
2,SQD,SQUID,88789
3,SLCD MNZNLL LVS SLD,SLICED MANZANILLA OLIVES SALAD,83595
4,PNPPL CHNKS TS WN JC,PINEAPPLE CHUNKS ITS OWN JUICE,70107


In [6]:
le.inverse_transform([8681])

array(['BISTRO STEAMERS PASTA PRIMAVERA'], dtype='<U185')

In [8]:
# X Vocab
MAX_SEQ_LEN = 25
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 256

flattened = np.asarray(x).reshape(-1)
t = Tokenizer(num_words=MAX_NUM_WORDS)
t.fit_on_texts(flattened)

## see encoded X --> do batch encoding, cannot load into memory
encoded_x = t.texts_to_sequences(flattened)
train_x = pad_sequences(encoded_x, maxlen=MAX_SEQ_LEN)
train_x

array([[   0,    0,    0, ...,  811,    2,  316],
       [   0,    0,    0, ...,    0,  550, 4097],
       [   0,    0,    0, ...,    0,    0, 1680],
       ...,
       [   0,    0,    0, ...,  176,  799,   66],
       [   0,    0,    0, ...,    0,   21,  675],
       [   0,    0,    0, ...,   70,  207,  474]])

In [24]:
model = Sequential()
model.add(Embedding(MAX_NUM_WORDS, EMBEDDING_DIM, input_length=MAX_SEQ_LEN))
model.add(LSTM(100))
model.add(Dense(max(df.encoded_y.tolist())+1, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 25, 256)           5120000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               142800    
_________________________________________________________________
dense_1 (Dense)              (None, 105722)            10677922  
Total params: 15,940,722
Trainable params: 15,940,722
Non-trainable params: 0
_________________________________________________________________
None


In [17]:
## train tokenizer outside, and pass in tokenizer so it doesn't change each time
x = df.X.tolist()
flattened = np.asarray(x).reshape(-1)
t = Tokenizer(num_words=MAX_NUM_WORDS)
t.fit_on_texts(flattened)


In [97]:
## test ##
def generate_examples(t, df):
   
    
    # X Vocab
    MAX_SEQ_LEN = 25
    MAX_NUM_WORDS = 20000
    EMBEDDING_DIM = 256
    
    x = df.X.tolist()
    flattened = np.asarray(x).reshape(-1)
    encoded_x = t.texts_to_sequences(flattened)
    train_x = pad_sequences(encoded_x, maxlen=MAX_SEQ_LEN)
    
    
    # Y Vocab
    y = df.encoded_y.tolist()
    num_classes_ = max(y) +1
    
    i = 0
    while True:
        gen_x = train_x[i:i+32]
        gen_y = to_categorical(y[i:i+32], num_classes=num_classes_)
        i+=32
        if i >= len(df) - 34:
            i = 0
        yield gen_x, gen_y

In [98]:
data_generator = generate_examples(t, df)
data_generator

<generator object generate_examples at 0x0000022E5E2F4EB8>

In [109]:
x_, y_ = next(data_generator)
for ea in y_:
    print (ea)
    print (np.argmax(ea))

[0. 0. 0. ... 0. 0. 0.]
40832
[0. 0. 0. ... 0. 0. 0.]
92999
[0. 0. 0. ... 0. 0. 0.]
59265
[0. 0. 0. ... 0. 0. 0.]
11612
[0. 0. 0. ... 0. 0. 0.]
20496
[0. 0. 0. ... 0. 0. 0.]
5351
[0. 0. 0. ... 0. 0. 0.]
13540
[0. 0. 0. ... 0. 0. 0.]
43380
[0. 0. 0. ... 0. 0. 0.]
37767
[0. 0. 0. ... 0. 0. 0.]
70249
[0. 0. 0. ... 0. 0. 0.]
962
[0. 0. 0. ... 0. 0. 0.]
19958
[0. 0. 0. ... 0. 0. 0.]
101098
[0. 0. 0. ... 0. 0. 0.]
82761
[0. 0. 0. ... 0. 0. 0.]
45969
[0. 0. 0. ... 0. 0. 0.]
95971
[0. 0. 0. ... 0. 0. 0.]
29208
[0. 0. 0. ... 0. 0. 0.]
64466
[0. 0. 0. ... 0. 0. 0.]
13891
[0. 0. 0. ... 0. 0. 0.]
8576
[0. 0. 0. ... 0. 0. 0.]
21348
[0. 0. 0. ... 0. 0. 0.]
79997
[0. 0. 0. ... 0. 0. 0.]
35040
[0. 0. 0. ... 0. 0. 0.]
32579
[0. 0. 0. ... 0. 0. 0.]
68563
[0. 0. 0. ... 0. 0. 0.]
37364
[0. 0. 0. ... 0. 0. 0.]
6081
[0. 0. 0. ... 0. 0. 0.]
9788
[0. 0. 0. ... 0. 0. 0.]
28095
[0. 0. 0. ... 0. 0. 0.]
88151
[0. 0. 0. ... 0. 0. 0.]
53929
[0. 0. 0. ... 0. 0. 0.]
15480


In [94]:
x_, y_ = next(data_generator)
print (x_.shape)
print (y_.shape)

(32, 25)
(32, 105722)


In [99]:
model.fit_generator(data_generator, steps_per_epoch=int(len(df)/32), epochs=10)

Epoch 1/10
16519/16519 [==============================] - 2776s 168ms/step - loss: 16.1179 - acc: 1.1351e-05
Epoch 2/10
16519/16519 [==============================] - 2852s 173ms/step - loss: 16.1179 - acc: 9.4588e-06
Epoch 3/10
16519/16519 [==============================] - 2852s 173ms/step - loss: 16.1179 - acc: 9.4588e-06
Epoch 4/10
16519/16519 [==============================] - 2852s 173ms/step - loss: 16.1179 - acc: 9.4588e-06
Epoch 5/10
16519/16519 [==============================] - 2850s 173ms/step - loss: 16.1179 - acc: 9.4588e-06
Epoch 6/10
16519/16519 [==============================] - 2853s 173ms/step - loss: 16.1179 - acc: 9.4588e-06
Epoch 7/10
16519/16519 [==============================] - 2855s 173ms/step - loss: 16.1179 - acc: 9.4588e-06
Epoch 8/10
16519/16519 [==============================] - 2852s 173ms/step - loss: 16.1179 - acc: 9.4588e-06
Epoch 9/10
16519/16519 [==============================] - 2856s 173ms/step - loss: 16.1179 - acc: 9.4588e-06
Epoch 10/10
16519/1

In [100]:
# start = time.time()
# model.fit(train_x, train_y, epochs=6)
model.save("lstm_generator_epochs_10.h5")
# print ("Took: ", time.time()-start)

# Evaluation

In [101]:
df.head()

,X,Y,encoded_y
0,MXCN SHRDDD CHS BLND,MEXICAN SHREDDED CHEESE BLEND,58983
1,VGTBL SHRTNNG,VEGETABLE SHORTENING,100452
2,SQD,SQUID,88789
3,SLCD MNZNLL LVS SLD,SLICED MANZANILLA OLIVES SALAD,83595
4,PNPPL CHNKS TS WN JC,PINEAPPLE CHUNKS ITS OWN JUICE,70107


In [102]:
test_num = 300
x = df.X.tolist()
flattened = np.asarray(x).reshape(-1)
encoded_x = t.texts_to_sequences(flattened)
train_x = pad_sequences(encoded_x, maxlen=MAX_SEQ_LEN)
train_x
# preds = model.predict(train_x[:test_num])
# pred_labels = [[np.argmax(x)] for x in preds]

array([[   0,    0,    0, ...,  811,    2,  316],
       [   0,    0,    0, ...,    0,  550, 4097],
       [   0,    0,    0, ...,    0,    0, 1680],
       ...,
       [   0,    0,    0, ...,  176,  799,   66],
       [   0,    0,    0, ...,    0,   21,  675],
       [   0,    0,    0, ...,   70,  207,  474]])

In [111]:
preds

array([[1.21526140e-08, 1.24887825e-08, 1.21448975e-08, ...,
        1.23546835e-08, 1.29347617e-08, 1.27887505e-08],
       [1.21526140e-08, 1.24887825e-08, 1.21448975e-08, ...,
        1.23546835e-08, 1.29347617e-08, 1.27887505e-08],
       [1.21526140e-08, 1.24887825e-08, 1.21448975e-08, ...,
        1.23546835e-08, 1.29347617e-08, 1.27887505e-08],
       ...,
       [1.21525678e-08, 1.24887345e-08, 1.21448744e-08, ...,
        1.23546835e-08, 1.29347129e-08, 1.27887008e-08],
       [1.21525678e-08, 1.24887345e-08, 1.21448744e-08, ...,
        1.23546835e-08, 1.29347129e-08, 1.27887008e-08],
       [1.21525678e-08, 1.24887345e-08, 1.21448744e-08, ...,
        1.23546835e-08, 1.29347129e-08, 1.27887008e-08]], dtype=float32)

In [103]:
pred_titles = le.inverse_transform(pred_labels)
pred_titles

c:\users\kevin\anaconda3\envs\tensorflow\lib\site-packages\sklearn\preprocessing\label.py:273: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array(['MEXICAN SHREDDED CHEESE BLEND', 'MEXICAN SHREDDED CHEESE BLEND',
       'MEXICAN SHREDDED CHEESE BLEND', 'MEXICAN SHREDDED CHEESE BLEND',
       'MEXICAN SHREDDED CHEESE BLEND', 'MEXICAN SHREDDED CHEESE BLEND',
       'MEXICAN SHREDDED CHEESE BLEND', 'MEXICAN SHREDDED CHEESE BLEND',
       'MEXICAN SHREDDED CHEESE BLEND', 'MEXICAN SHREDDED CHEESE BLEND',
       'MEXICAN SHREDDED CHEESE BLEND', 'MEXICAN SHREDDED CHEESE BLEND',
       'MEXICAN SHREDDED CHEESE BLEND', 'MEXICAN SHREDDED CHEESE BLEND',
       'MEXICAN SHREDDED CHEESE BLEND', 'MEXICAN SHREDDED CHEESE BLEND',
       'MEXICAN SHREDDED CHEESE BLEND', 'MEXICAN SHREDDED CHEESE BLEND',
       'MEXICAN SHREDDED CHEESE BLEND', 'MEXICAN SHREDDED CHEESE BLEND',
       'MEXICAN SHREDDED CHEESE BLEND', 'MEXICAN SHREDDED CHEESE BLEND',
       'MEXICAN SHREDDED CHEESE BLEND', 'MEXICAN SHREDDED CHEESE BLEND',
       'MEXICAN SHREDDED CHEESE BLEND', 'MEXICAN SHREDDED CHEESE BLEND',
       'MEXICAN SHREDDED CHEESE BLEND', 'MEXICAN SH

In [86]:
analysis_df = pd.DataFrame({'pred_x': pred_titles, 'x': x[:test_num], 'y': y[:test_num]})
analysis_df = analysis_df[['x', 'y', 'pred_x']]

In [87]:
analysis_df

,x,y,pred_x
0,MXCN SHRDDD CHS BLND,MEXICAN SHREDDED CHEESE BLEND,MEXICAN SHREDDED CHEESE BLEND
1,VGTBL SHRTNNG,VEGETABLE SHORTENING,MEXICAN SHREDDED CHEESE BLEND
2,SQD,SQUID,MEXICAN SHREDDED CHEESE BLEND
3,SLCD MNZNLL LVS SLD,SLICED MANZANILLA OLIVES SALAD,MEXICAN SHREDDED CHEESE BLEND
4,PNPPL CHNKS TS WN JC,PINEAPPLE CHUNKS ITS OWN JUICE,MEXICAN SHREDDED CHEESE BLEND
5,TLN STYL MTBLLS,ITALIAN STYLE MEATBALLS,MEXICAN SHREDDED CHEESE BLEND
6,YGRT RSNS SNCKS,YOGURT RAISINS SNACKS,MEXICAN SHREDDED CHEESE BLEND
7,BJ BLND SSND SWT CRN WTH RD ND GRN PPPR,BAJA BLEND SEASONED SWEET CORN WITH RED AND GR...,MEXICAN SHREDDED CHEESE BLEND
8,SWRL CRNCH SWTND WHL WHT ND RC CRL,SWIRL CRUNCH SWEETEND WHOLE WHEAT AND RICE CEREAL,MEXICAN SHREDDED CHEESE BLEND
9,PRSCTT CTT WTH HRBS FLLY CKD HM,PROSCIUTTO COTTO WITH HERBS FULLY COOKED HAM,MEXICAN SHREDDED CHEESE BLEND
